# Load Data & Packages

In [0]:
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division
import pandas as pd
import pandas_profiling as pf

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
from sklearn import ensemble

In [3]:
input = pd.read_csv('Train_UWu5bXk.csv')

FileNotFoundError: ignored

# Analyze & Clean Input

In [0]:
pf.ProfileReport(input)

Number of variables,12
Number of observations,8523
Total Missing (%),3.8%
Total size in memory,799.1 KiB
Average record size in memory,96.0 B
Numeric,5
Categorical,7
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [0]:
# Item_Fat_Content LF to Low Fat, what are other values? - OHE
input['Item_Fat_Content'] = input['Item_Fat_Content'].replace('LF','Low Fat')
input['Item_Fat_Content'] = input['Item_Fat_Content'].replace('low fat','Low Fat')
input['Item_Fat_Content'] = input['Item_Fat_Content'].replace('reg','Regular')
input['Item_Fat_Content'] = input['Item_Fat_Content'].astype('category')
input['low_fat'] = 1-(input['Item_Fat_Content'].cat.codes)

In [0]:
# Item_Type - OHE
input['Item_Type'].unique()
input['Item_Type'] = input['Item_Type'].astype('category')
input['Item_Type_Code'] = input['Item_Type'].cat.codes
#pf.ProfileReport(input[["Item_Type_Code"]])

In [0]:
# Item_Weight - Remove missing values (17.2%)
input['Item_Weight'] = input['Item_Weight'].fillna(0)
#pf.ProfileReport(input[["Item_Weight"]])

In [0]:
# Outlet_Identifier - OHE - just trying codes
input['Outlet_Identifier'] = input['Outlet_Identifier'].astype('category')
input['Outlet_ID'] = input['Outlet_Identifier'].cat.codes
#input['Outlet_ID'].head()

In [0]:
# Outlet_Location_Type - Codes
input['Outlet_Location_Type'].head(10)
input['Outlet_Location_Type'] = input['Outlet_Location_Type'].astype('category')
input['Outlet_Loc_Type_ID'] = input['Outlet_Location_Type'].cat.codes
#input['Outlet_Loc_Type_ID'].head()

In [0]:
# Outlet_Size - Remove missing values (28.3%)
input['Outlet_Size'] = input['Outlet_Size'].fillna('Unknown')
input['Outlet_Size'] = input['Outlet_Size'].astype('category')
input['Outlet_Size_ID'] = input['Outlet_Size'].cat.codes

In [0]:
# Outlet_Type - OHE
input['Outlet_Type'] = input['Outlet_Type'].astype('category')
input['Outlet_Type_ID'] = input['Outlet_Type'].cat.codes

In [0]:
input['Item_Identifier'] = input['Item_Identifier'].astype('category')
input['Item_ID'] = input['Item_Identifier'].cat.codes

In [0]:
input = pd.get_dummies(input, columns=["Outlet_Type"], prefix=["Outlet"])

In [0]:
input.dtypes

Item_Identifier              category
Item_Weight                   float64
Item_Fat_Content             category
Item_Visibility               float64
Item_Type                    category
Item_MRP                      float64
Outlet_Identifier            category
Outlet_Establishment_Year       int64
Outlet_Size                  category
Outlet_Location_Type         category
Item_Outlet_Sales             float64
low_fat                          int8
Item_Type_Code                   int8
Outlet_ID                        int8
Outlet_Loc_Type_ID               int8
Outlet_Size_ID                   int8
Outlet_Type_ID                   int8
Item_ID                         int16
Outlet_Grocery Store            uint8
Outlet_Supermarket Type1        uint8
Outlet_Supermarket Type2        uint8
Outlet_Supermarket Type3        uint8
dtype: object

In [0]:
final_input = input.select_dtypes(exclude=['object','category']).copy()

In [0]:
final_input.columns

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Item_Outlet_Sales', 'low_fat',
       'Item_Type_Code', 'Outlet_ID', 'Outlet_Loc_Type_ID', 'Outlet_Size_ID',
       'Outlet_Type_ID', 'Item_ID', 'Outlet_Grocery Store',
       'Outlet_Supermarket Type1', 'Outlet_Supermarket Type2',
       'Outlet_Supermarket Type3'],
      dtype='object')

# Create an Input Clean Up function

In [0]:
def input_cleanup(input):
    # Item_Fat_Content LF to Low Fat, what are other values? - OHE
    input['Item_Fat_Content'] = input['Item_Fat_Content'].replace('LF','Low Fat')
    input['Item_Fat_Content'] = input['Item_Fat_Content'].replace('low fat','Low Fat')
    input['Item_Fat_Content'] = input['Item_Fat_Content'].replace('reg','Regular')
    input['Item_Fat_Content'] = input['Item_Fat_Content'].astype('category')
    input['low_fat'] = 1-(input['Item_Fat_Content'].cat.codes)
    
    # Item_Type - OHE
    input['Item_Type'].unique()
    input['Item_Type'] = input['Item_Type'].astype('category')
    input['Item_Type_Code'] = input['Item_Type'].cat.codes

    # Item_Weight - Remove missing values (17.2%)
    input['Item_Weight'] = input['Item_Weight'].fillna(0)

    # Outlet_Identifier - OHE - just trying codes
    input['Outlet_Identifier'] = input['Outlet_Identifier'].astype('category')
    input['Outlet_ID'] = input['Outlet_Identifier'].cat.codes

    # Outlet_Location_Type - Codes
    input['Outlet_Location_Type'].head(10)
    input['Outlet_Location_Type'] = input['Outlet_Location_Type'].astype('category')
    input['Outlet_Loc_Type_ID'] = input['Outlet_Location_Type'].cat.codes

    # Outlet_Size - Remove missing values (28.3%)
    input['Outlet_Size'] = input['Outlet_Size'].fillna('Unknown')
    input['Outlet_Size'] = input['Outlet_Size'].astype('category')
    input['Outlet_Size_ID'] = input['Outlet_Size'].cat.codes

    # Outlet_Type - OHE
    input['Outlet_Type'] = input['Outlet_Type'].astype('category')
    input['Outlet_Type_ID'] = input['Outlet_Type'].cat.codes
    
    input['Item_Identifier'] = input['Item_Identifier'].astype('category')
    input['Item_ID'] = input['Item_Identifier'].cat.codes
    
    return input

In [0]:
features = ['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'low_fat',
       'Item_Type_Code', 'Outlet_ID', 'Outlet_Loc_Type_ID', 'Outlet_Size_ID','Outlet_Grocery Store',
       'Outlet_Supermarket Type1', 'Outlet_Supermarket Type2',
       'Outlet_Supermarket Type3']
output = 'Item_Outlet_Sales'

X = final_input[features]
y = final_input[output]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
degree = PolynomialFeatures(degree=3)

In [0]:
poly_X = degree.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=0)

In [0]:
poly_X.shape

(8523, 560)

In [0]:
def fit_score(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    print("Train Score:", model.score(X_train, y_train),"Test Score:",model.score(X_test, y_test))
    print("RMSE Train:",np.sqrt(mean_squared_error(model.predict(X_train),y_train)))
    print("RMSE Test:",np.sqrt(mean_squared_error(model.predict(X_test),y_test)))

In [0]:
lin_reg = LinearRegression()
fit_score(lin_reg, X_train, y_train, X_test, y_test)

Train Score: 0.6174354429696738 Test Score: 0.5984714258895377
RMSE Train: 1054.751389515308
RMSE Test: 1084.0745835606185


In [0]:
ridge_reg = linear_model.Ridge(alpha=0.5)
ridge_reg.fit(X_train,y_train)
print("Train Score:", ridge_reg.score(X_train, y_train),"Test Score:",ridge_reg.score(X_test, y_test))
print("RMSE Train:",np.sqrt(mean_squared_error(ridge_reg.predict(X_train),y_train)))
print("RMSE Test:",np.sqrt(mean_squared_error(ridge_reg.predict(X_test),y_test)))

In [0]:
ridge_reg_cv = linear_model.RidgeCV(alphas=range(10,60,), cv=3)
#ridge_reg_cv.fit(X_train,y_train)
print("Train Score:", ridge_reg_cv.score(X_train, y_train),"Test Score:",ridge_reg_cv.score(X_test, y_test))
print("RMSE Train:",np.sqrt(mean_squared_error(ridge_reg_cv.predict(X_train),y_train)))
print("RMSE Test:",np.sqrt(mean_squared_error(ridge_reg_cv.predict(X_test),y_test)))
print("Alpha Value:",ridge_reg_cv.alpha_)
#ridge_reg_cv.coef_

In [0]:
lasso_reg = linear_model.Lasso(alpha=700, max_iter=99999)
#lasso_reg.fit(X_train,y_train)
print("Train Score:", lasso_reg.score(X_train, y_train),"Test Score:",lasso_reg.score(X_test, y_test))
print("RMSE Train:",np.sqrt(mean_squared_error(lasso_reg.predict(X_train),y_train)))
print("RMSE Test:",np.sqrt(mean_squared_error(lasso_reg.predict(X_test),y_test)))
#lasso_reg.alpha_

In [0]:
lasso_reg_cv = linear_model.LassoCV(alphas=[750,1000,10000], cv=3, max_iter=10000)
#lasso_reg_cv.fit(X_train,y_train)
print("Train Score:", lasso_reg_cv.score(X_train, y_train),"Test Score:",lasso_reg_cv.score(X_test, y_test))
print("RMSE Train:",np.sqrt(mean_squared_error(lasso_reg_cv.predict(X_train),y_train)))
print("RMSE Test:",np.sqrt(mean_squared_error(lasso_reg_cv.predict(X_test),y_test)))
print(lasso_reg_cv.alpha_)
print(lasso_reg_cv.coef_)

In [0]:
min_leaves = range(110, 130, 2)

In [0]:
for min_leaf in min_leaves:
    tree_reg = tree.DecisionTreeRegressor(criterion='mse',min_samples_leaf=min_leaf)
    fit_score(tree_reg, X_train, y_train, X_test, y_test)
    print(min_leaf,tree_reg)

Train Score: 0.6202370856304444 Test Score: 0.5872973160339441
RMSE Train: 1050.8821516236308
RMSE Test: 1099.0553906787816
110 DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=110,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
Train Score: 0.6180029967994252 Test Score: 0.5899180835577089
RMSE Train: 1053.9687106599638
RMSE Test: 1095.5601916733976
112 DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=112,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
Train Score: 0.6178218731334642 Test Score: 0.5897539226775562
RMSE Train: 1054.2185503862202
RMSE Test: 1095.7794

In [0]:
rf_reg = ensemble.RandomForestRegressor(min_samples_leaf=100,n_jobs=4)
fit_score(rf_reg, X_train, y_train, X_test, y_test)
print(rf_reg)

Train Score: 0.6210524171720875 Test Score: 0.5879100904590884
RMSE Train: 1049.7534503834258
RMSE Test: 1098.2391575432896
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


In [0]:
gb_reg = ensemble.GradientBoostingRegressor(criterion='mse',min_samples_leaf=100)
fit_score(gb_reg, X_train, y_train, X_test, y_test)
print(gb_reg)

Train Score: 0.6441005913634039 Test Score: 0.5951945764268332
RMSE Train: 1017.3288797888118
RMSE Test: 1088.4891274365546
GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=100,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


In [0]:
ada_reg = ensemble.AdaBoostRegressor(n_estimators=1)
fit_score(ada_reg, X_train, y_train, X_test, y_test)
print(ada_reg)

Train Score: 0.5319527264919434 Test Score: 0.5104668295356493
RMSE Train: 1166.655411989888
RMSE Test: 1196.9940632493617
AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=1, random_state=None)


# Predicting Output

In [0]:
output_raw = pd.read_csv('Test_u94Q5KV.csv')

In [0]:
pf.ProfileReport(output_raw)

In [0]:
output_test = input_cleanup(output_raw)

In [0]:
output = degree.fit_transform(output_test[features])

In [0]:
predicted_output = lasso_reg_cv.predict(output)

In [0]:
final_submission = output_raw[['Item_Identifier','Outlet_Identifier']]

In [0]:
final_submission['Item_Outlet_Sales'] = predicted_output

In [0]:
final_submission.head()

In [0]:
final_submission.to_csv('Submission_v4.csv',index=False)